# Machine Learning
This is a long tutorial that will explore the basic applications of machine learning algorithms. The only part that is required to be turned in is the exercise section at the end. You are, of course, encouraged to look into the subject farther on your own but for this course, only a shallow understanding is expected/required to complete the activities.

# (1) Set Up

## First Things First
Imports

In [ ]:
import numpy as np # support for arrays and vectorized operations
import pandas as pd # support for DataFrames and Series
import matplotlib.pyplot as plt # main graphing library

import warnings
warnings.filterwarnings('ignore')

## Getting Sample Data
sklearn, the machine learning package we will be using has a number of toy datasets within it to show different models or compare algorithms. 

In [ ]:
from sklearn import datasets

iris = datasets.load_iris()
wine = datasets.load_wine()
diabetes = datasets.load_diabetes()

# (2) Preprocessing

It's important that we preprocess our data before we make machine learning techniques. We've already seen one type of preprocessing technique before, masking out NA values. Getting rid of NA values is essential as many ML algorithms can't handle them and will throw an error. 

There are two other types of preprocessing techniques which we will now go over:
1. Encoding - This means turning a categorical variable into a numeric veriable. ML algorithms can only work with numbers, and to allow it to understand qualitative variables, we convert them into numbers. The number itself doesn't matter. E.g. [Male, Female] could become [0, 1] or [1, 0] and the ML algorithm would treat them the same. Generally, the easiest way to do this is to just assign each of the possible values of the variable to a number sequentially, a method called Ordinal Encoding.
<br></br>
2. Scaling - This means reshaping the data to account for difference in distribution between variables. In this case, we are using standard scaling so that we remove the mean and scale the data to unit variance. What we end up with is a range of values preserve the relations between the original data points but is centered at 0.

Encoding may be a little easier to understand why it is necessary but to demonstrate why scaling is necessary, here is a short demo using dimensionality reduction techniques which we will explore late (separation of colors is the goal here).

## Scaling

In [ ]:
from sklearn.preprocessing import StandardScaler
from umap import UMAP
fix, axs = plt.subplots(1, 2, constrained_layout=True)

scaler = StandardScaler()

embedding_unscaled = UMAP().fit_transform(wine.data)

scaled_data = scaler.fit_transform(wine.data)
embedding_scaled = UMAP().fit_transform(scaled_data)

axs[0].scatter(embedding_unscaled[:, 0], embedding_unscaled[:, 1], c=wine.target, alpha=0.5, cmap='nipy_spectral')
axs[0].set_title('No Scaling')

axs[1].scatter(embedding_scaled[:, 0], embedding_scaled[:, 1], c=wine.target, alpha=0.5, cmap='nipy_spectral')
axs[1].set_title('Scaled Data')

## Encoding

In [ ]:
from sklearn.preprocessing import OrdinalEncoder

random_categorical = pd.DataFrame(
    {
    'gender' : ['male', 'male', 'female', 'male', 'female'],
    'smoking_status' : ['yes', 'no', 'yes', 'yes', 'no'],
    'age' : [36, 24, 57, 24, 13]
    }
)

random_categorical # sample dataset with categorical data

To encode (change from categorical to numerical) we can use OrdinalEncoder(). This works well when we have either a categorical variable with directional values or if we only have two possible values. Ensure that you are ONLY passing it the categorical columns (in this case, we DON'T want to pass it the age column)

In [ ]:
encoder = OrdinalEncoder()
unencoded_columns = random_categorical.loc[:, ['gender', 'smoking_status']]
encoded_columns = encoder.fit_transform(unencoded_columns)
random_categorical.loc[:, ['gender', 'smoking_status']] = encoded_columns

random_categorical

To switch back from numerical to categorical, we use the inverse function. Your encoder must be the original encoder model you used to first encode it, or it will not be able to revert it back.

In [ ]:
original_columns = encoder.inverse_transform(random_categorical.loc[:, ['gender', 'smoking_status']])
random_categorical.loc[:, ['gender', 'smoking_status']] = original_columns

random_categorical

# (3) Supervised Learning

## Train-Test Split
With supervised learning, we start with labeled data, split a portion of it off (test set) and use the other portion (train set) to train our model. We can then validate our model by using it on the test set and seeing how well it does.
<br></br>
sklearn makes this easy for us by providing a function which will do exactly that.

In [ ]:
from sklearn.model_selection import train_test_split

train_test_split takes three parameters
+ data - the data which will be split
+ target - the identified labels (must be numeric)
+ train_size - the percent which will be used to train versus test, commonly between 50/50 to 80/20, but the choice is ultimately up to you

In [ ]:
iris.target # 0, 1, 2 correspond to Iris satosa, Iris versicolour, Iris virginica 

In [ ]:
pd.DataFrame(iris.data, columns=iris.feature_names) # does not have to be in DF or have column names
# just in DF for ease of viewing

Note that X (capital) refers to the input dataframe and y (lowercase) refers to the labels, this notation is specific to supervised learning and should not be confused with the data x and y which we use for plotting.

train_test_split gives 4 outputs:
+ X_train - training data set
+ X_test - testing data set
+ y_train - training labels
+ y_test - testing labels

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, train_size=0.75)
print(X_train.shape) # dataset split by train size
print(X_test.shape) # dataset not included in above dataset
print(y_train.shape) # labels for corresponding X_train
print(y_test.shape) # labels for corresponding X_test

## Classification
There are many different classifier models. Here, we will explore Nearest Neighbors, one of the most widely used methods, as well as Decision Trees, Multilayer Perceptron, and Naive Bayes.
<br></br>
For more info, the sklearn website has a great implementation of visualizing different classifiers. https://scikit-learn.org/stable/auto_examples/classification/plot_classifier_comparison.html#sphx-glr-auto-examples-classification-plot-classifier-comparison-py
<br></br>
For a brief overview of each classifier:
+ **Nearest Neighbors** - Imagine we had a 2D dataset with an x axis and a y axis. For our training data, we would classify each point based on the its label. This is our model. For each point in the test set, we get the labels of the nearest points (from the training set) on our XY plane and classify it based on the labels of its neighbors. While a 2D plane is easy to visualize, Nearest Neighbors can be used in any number of dimensions/features.
<br></br>
+ **Decision Tree** - This classifier builds what is called a decision tree based on the data. Imagine a fork in the road with two possible routes to go, and then within each route is an additional fork each with their own additional forks and so on. Now instead of forks in the road, imagine a tree with different limbs that branch out the further you go from the trunk. The base of the tree is where the input goes in and based on the model, it winds up at the very end of a possible branch which is the output label. This is a good way to visualize how decision tree algorithms work.
<br></br>
+ **Multilayer Perceptron** - This is a neural network based classifier. A neural network with a predetermined number of intermediate layers is created based on the training data.
<br></br>
+ **Naive-Bayes** - This is algorithm based on the independence clause in Bayes' Theorem. Each feature (column/variable) is assumed to be independent of every other feature. A probabilistic model is then built around this to arrive at the most likely output for every given input.

In [ ]:
from sklearn.neighbors import KNeighborsClassifier # default number of neighbors looked at is 5
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB

Basic Skeleton:

In [ ]:
# Pre-Process Data First!!!
classifier = FILL IN # substitute with any classifier
classifier.fit(X_train, y_train) # trains model to training dataset, labels
y_pred = classifier.predict(X_test) # uses modelto predict labels of X_test

Doing this yields the array y_pred which is an array of numeric labels that the model has decided. You can compare this with y_test to see the accuracy.

In [ ]:
sum(y_pred == y_test) / len(y_test) # percent correct

Example using KNeighborsClassifier

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, train_size=0.8)

classifier = KNeighborsClassifier()
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)

accuracy = sum(y_pred == y_test) / len(y_test)

print(f'Using the KNeighborsClassifier, the model was able to accurately predict {accuracy * 100}% of the test set.')

## Regression

Regression is very much like classification however instead of starting with numeric data and ending with a categorical label, we start with numeric data and end with a numeric output.
<br></br>
We will be exploring Linear Regression, perhaps the simplest model, as well as Nearest Neighbors Regression, Stochastic Gradient Descent Regression, Support Vector Machine, and Bayesian Ridge Regression.
<br></br>
Due to the complex math behind each of these models, we will not go into details but the sklearn website is a great resource for further details.

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import SGDRegressor
from sklearn.svm import SVR
from sklearn.linear_model import BayesianRidge

The code behind regression is the exact same as in classification (SEE ABOVE) except instead of y_pred being a collection of numerical labels, it will be a numerical quantitative output.

In [ ]:
scaler = StandardScaler()
scaled_data = scaler.fit_transform(diabetes.data)

regressors_names = [
    'Linear', 
    'Nearest Neighbors', 
    'Stochastic Gradient Descent', 
    'Support Vector',
    'Bayesian Ridge'
]

regressors = [
    LinearRegression(),
    KNeighborsRegressor(),
    SGDRegressor(),
    SVR(),
    BayesianRidge()
]

regressors_perf = {
    0: [],
    1: [],
    2: [],
    3: [],
    4: []
}

for n in range(1000):
    X_train, X_test, y_train, y_test = train_test_split(scaled_data, diabetes.target, train_size=0.8)
    for i in range(len(regressors)):
        model = regressors[i]
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        difference = y_pred - y_test
        regressors_perf[i].append(abs(np.mean(difference)))
    if n % 50 == 0:
        print(f'Completed {n}/1000 simulations')
        
print('\nAfter 1000 simulations, the average error for each regressor is as follows:')
for i in regressors_perf:
    print(f'\t{regressors_names[i]} : {np.mean(regressors_perf[i])}')



# (4) Unsupervised Learning

## Dimensionality Reduction
Dimensionality reduction is a technique used to take a dataset with a lot of different dimensions (e.g. columns/features) and reduce each data point down to a lower dimensional space. In this case, we will be reducing our data down to 2D so we can plot it on the XY plane. It's important to note that when dimensions are reduced, the X and Y axis don't represent any specific feature, but rather represent alikeness across every dimension/feature.
<br></br>
The three methods we will be using are in order of their complexity and development data. All three are established in research.
+ Principal Component Analysis (PCA)
+ t-Distributed Stochastic Neighbor Embedding (t-SNE)
+ Uniform Manifold Approximation and Projection (UMAP) 

In [ ]:
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from umap import UMAP

### Skeleton Code

In [ ]:
# Pre-Process Data First!!!
reducer = FILL IN
embedding = reducer.fit_transform(DATA)

### Plotting

In [ ]:
x_vals = embedding[:, 0]
y_vals = embedding[:, 1]

Demonstration

In [ ]:
plt.rcParams['figure.figsize'] = [12, 5]

scaler = StandardScaler()
scaled_data = scaler.fit_transform(wine.data)

reducers = [
    PCA(),
    TSNE(),
    UMAP()
]
reducers_names = ['PCA', 't-SNE', 'UMAP']

fig, axs = plt.subplots(1, 3, constrained_layout=True)

for i in range(3):
    embedding = reducers[i].fit_transform(scaled_data)
    x_vals = embedding[:, 0]
    y_vals = embedding[:, 1]
    axs[i].scatter(x_vals, y_vals, c=wine.target, cmap='nipy_spectral', alpha=0.5)
    axs[i].set_title(reducers_names[i])


## Clustering
There are a wide variety of different clustering algorithms. Here, we will only examine K-Means, perhaps the most commonly used clustering algorithm, and HDBSCAN, an improved version of the standard DBSCAN algorithm, which I think is one of the best if the number of clusters is unknown :)
<br></br>
But here is a link to the sklearn implementation of different clustering algorithms with some helpful visualizations of the differences between each algorithm.
https://scikit-learn.org/stable/modules/clustering.html

In [ ]:
from sklearn.cluster import KMeans
from yellowbrick.cluster import KElbowVisualizer
from hdbscan import HDBSCAN

### K-Means
By default, K-Means will attempt to make a guess as to how many clusters there are based on similarity in location. The number of clusters then, will vary. You can however manually set the number of clusters using the n_clusters parameter.

In [ ]:
fig, axs = plt.subplots(1, 3, constrained_layout=True)
plt.rcParams['figure.figsize'] = [12, 5]

scaler = StandardScaler()
scaled_data = scaler.fit_transform(wine.data)

reducer = PCA() # arbitrary for this demo
embedding = reducer.fit_transform(scaled_data)
x_vals = embedding[:, 0]
y_vals = embedding[:, 1]

param_names = ['No Params', 'Clusters = 3', 'Clusters = 15']
cluster_models = [
    KMeans(),
    KMeans(n_clusters=3),
    KMeans(n_clusters=15)
]

for i in range(3):
    labels = cluster_models[i].fit_predict(embedding)
    
    axs[i].scatter(x_vals, y_vals, c=labels, cmap='nipy_spectral', alpha=0.5)
    axs[i].set_title(param_names[i])

### Elbow Plots
How do we figure out whow many clusters to use for KMeans? One way to do this is to create an elbow plot. An elbow plot shows a metric called inertia for various different possible K values. A higher K means a lower inertia score the lower the inertia score the better, but there is a point of diminishing returns. Our aim is to find the number of clusters, K, that results in the best clusters/inertia ratio. You can find this by visualizing the plot as an arm and trying to find the spot that is the elbow.

In [ ]:
# elbow plot method 1
scaler = StandardScaler()
scaled_data = scaler.fit_transform(wine.data)

wcss = []
for i in range(1, 11):
    kmeans = KMeans(n_clusters=i, init='k-means++', max_iter=300, n_init=10, random_state=0)
    kmeans.fit(scaled_data)
    wcss.append(kmeans.inertia_)
plt.plot(range(1, 11), wcss)
plt.title('Elbow Method')
plt.xlabel('Number of clusters')
plt.ylabel('WCSS')
plt.show()

yellowbrick, another package, provides a very convenient visualizer function which automatically graphs an elbow plot and tells you what the best number of clusters is.

In [ ]:
# elbow  plot method 2
visualizer = KElbowVisualizer(KMeans(), k=(1,11))
visualizer.fit(scaled_data)

### Hierarchical Density-Based Spatial Clustering of Applications with Noise
"HDBSCAN is a clustering algorithm developed by Campello, Moulavi, and Sander. It extends DBSCAN by converting it into a hierarchical clustering algorithm, and then using a technique to extract a flat clustering based in the stability of clusters."
https://hdbscan.readthedocs.io/en/latest/how_hdbscan_works.html
<br></br>
Unlike K-Means clustering, HDBSCAN does not allow you to choose the default number of clusters, but if you do not know the exact amount of clusters, it generally performs better than other algorithms. In addition, unlike K-Means clustering, HDBSCAN does not need to assign every value to a label, and you can see the label -1 used when it is not confident about which cluster the data belongs to (this is shown as dark grey on the plot).
<br></br>
Due to the method in which the algorithm works, HDBSCAN is not a good choice for clustering based on PCA.

In [ ]:
plt.rcParams['figure.figsize'] = [12, 5]

scaler = StandardScaler()
scaled_data = scaler.fit_transform(wine.data)

cluster_model = HDBSCAN()

reducers = [
    PCA(),
    TSNE(),
    UMAP()
]
reducers_names = ['PCA', 't-SNE', 'UMAP']

fig, axs = plt.subplots(1, 3, constrained_layout=True)

for i in range(3):
    embedding = reducers[i].fit_transform(scaled_data)
    x_vals = embedding[:, 0]
    y_vals = embedding[:, 1]
    labels = cluster_model.fit_predict(embedding)
    axs[i].scatter(x_vals, y_vals, c=labels, cmap='nipy_spectral', alpha=0.5)
    axs[i].set_title(reducers_names[i])
    

# (5) Sidenote on Hyperparameters
Nearly every machine learning has a variety of different variables to finetune the performance. Certain algorithms are better than others using the default parameters, but to find the absolute best one, you have to play around. These variables are called hyperparameters and there is good documentation on the scikit-learn/UMAP/HDBSCAN websites. 
<br></br>
There are also a number of functions within sklearn to find the best parameter. This will not be covered in this course and you are not expected to use them, but if you are interested, here is a link to their documentation on finding the best model: https://scikit-learn.org/stable/model_selection.html#model-selection

# (6) Implementation With -Omics Data
Omics data works the exact same as the other data in this tutorial. Just remember to identify which machine learning technique to use, and to use every preprocessing step we talked about:
+ Clean NA values
+ Encode categorical variables into numeric labels
+ Scale data

# (7) Exercises - only this part will be graded
Exercises 1 & 2 will be graded on accuracy. Exercise 3 will be graded on completion and effort. No extra credit will be granted outside of Exercise 4.

**Exercise 1** - Preprocessing/Supervised Learning (3 pt)
1. Fill in the skeleton code to complete the iris classification plot.
2. Answer the following questions:
+ Define the following preprocessing methods and describe why each are necessary
    + NA removal
    + Scaling
    + Encoding
+ What is the difference between classification and regression?

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn import datasets

iris = datasets.load_iris()

unscaled_data = iris.data

scaler = StandardScaler()
scaled_data = FILL IN HERE

classifier = FILL IN HERE # your choice of which classifier to use

FILL IN HERE = train_test_split(scaled_data, iris.target, train_size=0.75)

classifier.fit(FILL IN HERE)

y_pred = FILL IN HERE

Visualize your predictions here

In [ ]:
import matplotlib.pyplot as plt
fig, axs = plt.subplots(2, 2)

iris_df = pd.DataFrame(X_test)

axs[0][0].scatter(iris_df.iloc[:, 0], iris_df.iloc[:, 1], c=y_pred, s=20, cmap='gist_rainbow')
axs[0][0].set_ylabel('sepal width (cm)')

axs[0][1].scatter(iris_df.iloc[:, 2], iris_df.iloc[:, 1], c=y_pred, s=20, cmap='gist_rainbow')

axs[1][0].scatter(iris_df.iloc[:, 0], iris_df.iloc[:, 3], c=y_pred, s=20, cmap='gist_rainbow')
axs[1][0].set_xlabel('sepal length (cm)')
axs[1][0].set_ylabel('petal width (cm)')

axs[1][1].scatter(iris_df.iloc[:, 2], iris_df.iloc[:, 3], c=y_pred, s=20, cmap='gist_rainbow')
axs[1][1].set_xlabel('petal length (cm)')

fig.suptitle('Comparing Length and Width Between Sepals and Petals')

In [ ]:
# answer questions here

**Exercise 2** - Unsupervised Learning (2 pt)
1. Fill in the skeleton code to complete the iris dimensionality reduction and clustering plot.
2. Answer the following questions:
+ What do the axes on a UMAP, t-SNE, or PCA plot represent?
+ Compare the clustered plot vs the actual labels, how close were the clustered labels?

In [ ]:
# Step 1) Dimensionality Reduction
from umap import UMAP

reducer = UMAP()

embedding = FILL IN HERE # use the scaled_data from exercise 1 for the data

plt.scatter(embedding[:, 0], embedding[:, 1], color='purple')

In [ ]:
# Step 2) Determination of # of Clusters for KMeans()
from sklearn.cluster import KMeans
from yellowbrick.cluster import KElbowVisualizer

visualizer = KElbowVisualizer(FILL IN HERE)
visualizer.fit(FILL IN HERE)

In [ ]:
# Step 3) KMeans Clustering
cluster_model = KMeans(n_clusters = FILL IN HERE)

labels = cluster_model.fit_predict(embedding)

fig, axs = plt.subplots(1, 2, constrained_layout=True, sharey=True)

axs[0].scatter(embedding[:, 0], embedding[:, 1], c=labels, cmap='nipy_spectral', alpha=0.5)
axs[0].set_title('Clustered')
axs[1].scatter(embedding[:, 0], embedding[:, 1], c=iris.target, cmap='gnuplot', alpha=0.5)
axs[1].set_title('Actual Labels')

In [ ]:
# answer questions here

**Exercise 3** - Do It Yourself (5 pt)
<br></br>dermatology.csv is a CSV file with 34 different features. The exact description of each feature is in the dermatology.txt file. Each patient has one of the following conditions (this is the 35th column):
+ psoriasis (1)
+ seboreic dermatitis (2)
+ lichen planus (3)
+ pityriasis rosea (4)
+ chronic dermatitis (5)
+ pityriasis rubra pilaris (6)

Part I - Preprocessing (1 pt)
1. Load the data in
2. Separate the target column
3. Preprocess the data using necessary methods

In [ ]:
# write code here

Part II - Supervised Learning (2 pt)
1. Create a train-test split with an appropriate split-ratio
2. Test at least 3 different classification techniques to determine which is best able to predict skin condition and present it in a graphical method (can be separate or joined figures)
3. Use the age column as a target and create a regression model to predict age from the other features (include skin condition as a feature)

In [ ]:
# write code here

Part III - Unsupervised Learning (2 pt)
1. Use all three dimensionality-reduction techniques to show the data from Part I in 2D (can be separate or joined figures) Pick the dimensionality-reduction technique which results in the greatest separation
2. Create an elbow plot to determine the K number of clusters for K-Means
3. Cluster based on K-Means and HDBSCAN in a figure which shows both methods side by side

In [ ]:
# write code here

Part IV - Brief Response (REQUIRED IN ORDER TO GET CREDIT FOR EACH OTHER PART)
<br></br>
Answer the following questions if you completed the part:
1. Which of the preprocessing techniques were required for the data? How did you know?
2. Which supervised learning technique (classification vs regression) was best able to predict its respective target? Why do you think so?
3. Interpret what the distance between each point represents on the reduced dimensions figure.

In [ ]:
# answer questions here

**Exercise 4** - Challenge - Extra Credit (5 pts possible) <br>
</br>
INDIVIDUAL WORK ONLY - credit will be awarded based on thoughtful completion and demonstration of an understanding of ML principles, <u>code comments must be included</u>
<br></br>mushroom.csv is a dataset includes 61069 hypothetical mushrooms with caps based on 173 species (353 mushrooms per species). Each mushroom is identified as edible or poisonous. Of the 20 variables, 17 are nominal and 3 are metrical. The features and dataset design can be found in mushroom.txt. Your task is as follows:
+ Load and process the data (one challenge we often find ourselves facing is having to reformat data, this will be an exercise in that) from the .csv file
+ Create a classifier design and test that it can correctly identify whether a mushroom is edible or poisonous with a minimum accuracy of no lower than 95%. You will determine this by testing it on a 60/40 train-test split 20 times.

In [ ]:
# write code here